# Getting object types and positions
### Though we will be doing star/galaxy classification with ZTF data, they do not have type labels associated with objects (i.e. you don't know whether something is a star or galaxy). In this notebook, we show how to crossmatch with another survey (in this case, the Legacy Survey), to get the sky positions for our labeled truth data. These positions can then be fed into ZTF to make the cutouts for classification.

More information about the Legacy Survey: https://www.legacysurvey.org

In [ ]:
import astropy.io.fits as fits
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import search_around_sky, SkyCoord
from astropy import units as u

### Import the data using the specified URL for the full data you would like to use. 

We use a sweep catalog, returning all objects between ra = (0, 10) degrees and dec = (-5, 0) degrees.

More information on Legacy Survey sweep files: https://www.legacysurvey.org/dr7/files/#sweep-catalogs

In [ ]:
target_url = 'https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr9/south/sweep/9.0/sweep-000m005-010p000.fits'

In [ ]:
data_raw = fits.getdata(target_url, header=True)
data = data_raw[0]

### Save all galaxy types as 'gals', and all stars as 'stars'

In [ ]:
#"REX"=round exponential galaxy, "DEV"=deVauc, "EXP"=exponential, "SER"=Sersic
gal_mask = (data['type'] == 'REX') | (data['type'] == 'DEV') | (data['type'] == 'EXP') | (data['type'] == 'SER')
gals = data[gal_mask]
print("# of all galaxies in catalog: ", len(gals))

In [ ]:
#"PSF"=stellar
psf_mask = (data['TYPE'] == 'PSF')
stars = data[psf_mask]
print("# of all stars in catalog: ", len(stars))

### Optional "right ascension" (RA) and "declination" (DEC) cut:
RA and DEC are to the sky, what longitutde and lattitude are to the earth. A helpful introduction to RA and DEC can be found here:
https://skyandtelescope.org/astronomy-resources/right-ascension-declination-celestial-coordinates/

You may want objects only within a certain ra and dec region within your file. Right now we have a region that is 10° in RA and 5° in DEC. Let's make it a 5° by 5° region with a cut to RA:

In [ ]:
gals = gals[(gals['RA'] > 5) & (gals['RA'] < 10)]
stars = stars[(stars['RA'] > 5) & (stars['RA'] < 10)]

### Magnitude cuts:

Astronomers make magnitude cuts to objects according to the *"limiting magnitudes"* of the telecope. Limiting magnitude is essentially the magnitude of the faintest object the telescope and detect. Flux and magnitude are mathematically related and describe the "brightness" of the object. More information can be found here: https://www.e-education.psu.edu/astro801/content/l4_p5.html. (It is important to note that the magnitude system is reversed! i.e. a lower magnitude corresponds to a brighter object)

The Legacy Survey suppplies us with fluxes that must be converted to magnitudes. Doing so involves taking the log of the flux, so we need to make sure there are no negative flux values. (Note: negative flux is possible in very rare cases where you know an object is there but aren't seeing it in the current image, etc.)

In [ ]:
# Ensure all fluxes > 0

gals = gals[(gals['FLUX_G'] > 0) & (gals['FLUX_R'] > 0) & (gals['FLUX_Z'] > 0)]
stars = stars[(stars['FLUX_G'] > 0) & (stars['FLUX_R'] > 0) & (stars['FLUX_Z'] > 0)]

Once there are no negative values, flux can be converted to magnitude using:
$Magnitude = 22.5-2.5log_{10}(Flux)$

In [ ]:
# Calculate the magnitudes from the fluxes

gal_mag_g = 22.5-2.5*np.log10(np.array(gals['FLUX_G']))
gal_mag_r = 22.5-2.5*np.log10(np.array(gals['FLUX_R']))
gal_mag_z = 22.5-2.5*np.log10(np.array(gals['FLUX_Z']))

star_mag_g = 22.5-2.5*np.log10(np.array(stars['FLUX_G']))
star_mag_r = 22.5-2.5*np.log10(np.array(stars['FLUX_R']))
star_mag_z = 22.5-2.5*np.log10(np.array(stars['FLUX_Z']))

Because we will be obtaining object positions from Legacy, and then using those positions to get their images from ZTF, we want to make cuts on the ZTF limiting magnitudes, to ensure ZTF can detect them!

The limiting magnitudes for ZTF are: $g<20.8$, $r<20.6$, and $i<19.9$
This means that in the g-band, it can detect objects as faint as magnitude 20.8, and so-on. 

More info on photometric bands here: https://en.wikipedia.org/wiki/Photometric_system

***Note***: ZTF uses the g, r, and i bands, while the Legacy Survey consists of g-band, r-band, and z-band images. The further down the list the band, less faint objects can be detect (*usually*). We can roughly estimate that if ZTF can detect i-band to magnitude 19.9, it can probably detect in z-band to *roughly* 19.5. For the g-band and r-band we also round down a bit as we don't want to test the classifier on the faintest objects, *yet*.

In [ ]:
# Make a magnitude cut to satisfy (and go a little bit under) the limiting magnitude cutoffs for ZTF

gals = gals[(gal_mag_g < 20.5) & (gal_mag_r < 20) & (gal_mag_z < 19.5)]
stars = stars[(star_mag_g < 20.5) & (star_mag_r < 20) & (star_mag_z < 19.5)]

print("# of galaxies after magnitude cuts: ", len(gals))
print("# of stars after magnitude cuts: ", len(stars))

### Basic blend cut:

A "blend" simply means that there are two objects overlapping each other in the line-of-sight. Because the classifier uses "cutouts" (each image consists of just one object), we don't want any images with more than one object within a certain amount of space!

You can first make a "basic" blend cut but cutting on "fractional flux". This value can tell you if there is flux contribution from other sources nearby. At the two extremes: a value of 0 means there is no flux from objects nearby, and a value of 1 means ALL flux is from objects nearby. 

***Note:*** We again do a cut to ensure no negative frac flux values make their way in. (These can be caused by objects with negative flux or objects with a neighbor with negative flux)

In [ ]:
# Discard objects with high FRACFLUX's, as they are blends

gals = gals[(gals['FRACFLUX_G'] <= .1) & (gals['FRACFLUX_G'] >= 0) & (gals['FRACFLUX_R'] <= .1) & (gals['FRACFLUX_R'] >= 0) & (gals['FRACFLUX_Z'] <= .1) & (gals['FRACFLUX_Z'] >= 0)]
stars = stars[(stars['FRACFLUX_G'] <= .1) & (stars['FRACFLUX_G'] >= 0) & (stars['FRACFLUX_R'] <= .1) & (stars['FRACFLUX_R'] >= 0) & (stars['FRACFLUX_Z'] <= .1) & (stars['FRACFLUX_Z'] >= 0)]

print("# of galaxies after basic blend cuts: ", len(gals))
print("# of stars after basic blend cuts: ", len(stars))

### More sophisticated blend cut:

One can then do a better blend cut by compairing each objects RA and DEC to each other objects RA and DEC and remove every object that is within a certain distance of another object. 

We are making square cutouts of 20" by 20". ***Note:*** The double quote here means "arc second", where $1"=\frac{1}{3600}°$

Astropy's `search_around_sky` searches in a circular region. If our square is 20" x 20", we need a circle that can inscribe that square, to ensure no objects are inside that square. Our search region radius becomes $10\sqrt{2}$ arseconds.

Once we determine which objects are within a cutout width of eachother, we cut them out!


In [ ]:
# Some data manipulation for changing data from a fits recarray to a pandas DataFrame
gal_ra = gals['RA'].byteswap().newbyteorder()
gal_dec = gals['DEC'].byteswap().newbyteorder()
gal_type = gals['TYPE'].byteswap().newbyteorder()

star_ra = stars['RA'].byteswap().newbyteorder()
star_dec = stars['DEC'].byteswap().newbyteorder()
star_type = stars['TYPE'].byteswap().newbyteorder()

In [ ]:
# Save data as pandas DataFrame for using astropy
data = pd.DataFrame({'ra': np.concatenate((gal_ra, star_ra)), 'dec': np.concatenate((gal_dec, star_dec)), 'type': np.concatenate((gal_type, star_type))})

In [ ]:
# Define our coordinate system
c = SkyCoord(ra=data['ra']*u.degree, dec=data['dec']*u.degree)

# Use search_around_sky to see if any two objects are within half a cutout width of each other
idx1, idx2, sep2, dist3 = search_around_sky(c, c, (10*np.sqrt(2))*u.arcsec)

In [ ]:
# Get a list of unique indexes where blends occur
idx = (idx1 != idx2)
idx1 = idx1[idx]

In [ ]:
# Drop these rows from our dataframe
data.drop(data.index[idx1], axis=0, inplace=True)

In [ ]:
# Seperate back into galaxies and stars for easier writing to .csv
gal_mask2 = (data['type'] == 'REX') | (data['type'] == 'DEV') | (data['type'] == 'EXP') | (data['type'] == 'SER')
psf_mask2 = (data['type'] == 'PSF')

gals = data[gal_mask2]
stars = data[psf_mask2]

print("# of galaxies after sophisticated blend cuts: ", len(gals))
print("# of stars after sophisticated blend cuts: ", len(stars))

We now have a complete dataset of stars and galaxies that are:
- Within the ra and dec range we want
- Have magnitudes that are detectable with ZTF
- Should not be blended with another object

### Randomly sample 10k galaxies and 10k stars, and save their RA, DEC, and truth labels to a .csv for use in the next step!

In [ ]:
# Get data into a pandas dataframe for easy writing to .csv
gals = pd.DataFrame({'ra': gals['ra'], 'dec': gals['dec'], 'type': gals['type']})
stars = pd.DataFrame({'ra': stars['ra'], 'dec': stars['dec'], 'type': stars['type']})

# Sample 10k of each object
final_gals = gals.sample(10000)
final_stars = stars.sample(10000)

In [ ]:
final_stars.to_csv('stars.csv')
final_gals.to_csv('gals.csv')